# Здесь мы парсим xml

In [29]:
# регулярки.
import re
# удобная штука для того, чтобы понимать, сколько осталось в цикле.
from tqdm import tqdm
# pretty print
from pprint import pprint
# токенизация
from nltk import sent_tokenize

Короче, я в замешательстве. Сейчас экспериментирую только с одним файлом:

In [71]:
with open('xml_data/Война и мир. 9.xml', encoding='utf-8') as f:
    text = f.read()

Вот так примерно выглядит этот текст:

In [112]:
print(text[:6000])

<?xml version='1.0' encoding='utf-8'?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:xi="http://www.w3.org/2001/XInclude">
<teiHeader>
<fileDesc>
<titleStmt>
<title> Война и мир. </title>
<author> Толстой Л.Н. </author>
<respStmt>
<resp> подготовка TEI/XML </resp>
<name> Мария Картышева, Евгений Можаев, Даниил Скоринкин, Елена Сидорова </name>
</respStmt>
<respStmt>
<name> Анастасия Бонч-Осмоловская, Фёкла Толстая, Борис Орехов </name>
<resp> Идея, постановка задач, руководство </resp>
</respStmt>
</titleStmt>
<publicationStmt>
<publisher>
<orgName> Школа лингвистики НИУ ВШЭ </orgName>
</publisher>
<availability>
<p> Тексты и метатекстовая разметка доступны для свободного использования и распространения по
лицензии Creative Commons Attribution Share-Alike (cc by-sa) </p>
</availability>
</publicationStmt>
<sourceDesc>
<biblStruct>
<analytic>
<author> Толстой Л.Н. </author>
<title level="a"> Война и мир. </title>
</analytic>
<monogr>
<title level="m"> Полное собрание сочинений. Серия п

А вот так он выглядит для компа на самом деле:

In [113]:
text[:6000]

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n<teiHeader>\n<fileDesc>\n<titleStmt>\n<title> Война и мир. </title>\n<author> Толстой Л.Н. </author>\n<respStmt>\n<resp> подготовка TEI/XML </resp>\n<name> Мария Картышева, Евгений Можаев, Даниил Скоринкин, Елена Сидорова </name>\n</respStmt>\n<respStmt>\n<name> Анастасия Бонч-Осмоловская, Фёкла Толстая, Борис Орехов </name>\n<resp> Идея, постановка задач, руководство </resp>\n</respStmt>\n</titleStmt>\n<publicationStmt>\n<publisher>\n<orgName> Школа лингвистики НИУ ВШЭ </orgName>\n</publisher>\n<availability>\n<p> Тексты и метатекстовая разметка доступны для свободного использования и распространения по\nлицензии Creative Commons Attribution Share-Alike (cc by-sa) </p>\n</availability>\n</publicationStmt>\n<sourceDesc>\n<biblStruct>\n<analytic>\n<author> Толстой Л.Н. </author>\n<title level="a"> Война и мир. </title>\n</analytic>\n<monogr>\n<title level="m

По пути я решил в итоге проблему (т.е. какую регулярку нужно искать, чтобы он находил не строки, а абзацы).

In [115]:
text_paragraphs = re.findall(r'<p rend="left">(.*?)<\/p>', text, re.DOTALL)

Я сначала думал, что каждая строчка в кавычках -- это отдельный элемент, но только сейчас заметил, что между ними нет запятых.

In [123]:
pprint(text_paragraphs[:10])

[' — Eh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des\n'
 'поместья, de la famille Buonaparte. Non, je vous préviens, que si vous ne me '
 'dites pas, que\n'
 'nous avons la guerre, si vous vous permettez encore de pallier toutes les '
 'infamies, toutes les\n'
 'atrocités de cet Antichrist (ma parole, j’y crois) — je ne vous connais '
 'plus, vous n’êtes plus\n'
 'mon ami, vous n’êtes plus мой верный раб, comme vous dites. <note n="n1">\n'
 '<div type="section" xml:id="n1">\n'
 '<head>\n'
 '<ref target="index.xhtml#backn1"> 1 </ref>\n'
 '</head>\n'
 '<p rend="left"> Ну, князь, Генуя и Лукка поместья фамилии Бонапарте. Нет, я '
 'вам вперед\n'
 'говорю, если вы мне не скажете, что у нас война, если вы еще позволите себе '
 'защищать все\n'
 'гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист), — я '
 'вас больше не\n'
 'знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите. ',
 ' Я вижу, что я вас пугаю. ',
 ' Так говорила в июле 18

Оказывается, что если принтить один элемент, то он вот такой:

In [124]:
pprint(text_paragraphs[0])

(' — Eh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des\n'
 'поместья, de la famille Buonaparte. Non, je vous préviens, que si vous ne me '
 'dites pas, que\n'
 'nous avons la guerre, si vous vous permettez encore de pallier toutes les '
 'infamies, toutes les\n'
 'atrocités de cet Antichrist (ma parole, j’y crois) — je ne vous connais '
 'plus, vous n’êtes plus\n'
 'mon ami, vous n’êtes plus мой верный раб, comme vous dites. <note n="n1">\n'
 '<div type="section" xml:id="n1">\n'
 '<head>\n'
 '<ref target="index.xhtml#backn1"> 1 </ref>\n'
 '</head>\n'
 '<p rend="left"> Ну, князь, Генуя и Лукка поместья фамилии Бонапарте. Нет, я '
 'вам вперед\n'
 'говорю, если вы мне не скажете, что у нас война, если вы еще позволите себе '
 'защищать все\n'
 'гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист), — я '
 'вас больше не\n'
 'знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите. ')


И он отлично токенизирует предложения:

In [125]:
sent_tokenize(text_paragraphs[0])

[' — Eh bien, mon prince.',
 'Gênes et Lucques ne sont plus que des apanages, des\nпоместья, de la famille Buonaparte.',
 'Non, je vous préviens, que si vous ne me dites pas, que\nnous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les\natrocités de cet Antichrist (ma parole, j’y crois) — je ne vous connais plus, vous n’êtes plus\nmon ami, vous n’êtes plus мой верный раб, comme vous dites.',
 '<note n="n1">\n<div type="section" xml:id="n1">\n<head>\n<ref target="index.xhtml#backn1"> 1 </ref>\n</head>\n<p rend="left"> Ну, князь, Генуя и Лукка поместья фамилии Бонапарте.',
 'Нет, я вам вперед\nговорю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все\nгадости, все ужасы этого Антихриста (право, я верю, что он Антихрист), — я вас больше не\nзнаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите.']

Сейчас работаю над тем, чтобы теги внутри абзацев исчезли. Ну и хочу, чтобы он потом выловил из всех абзацев только реплики, а потом распределил среди них русские и французские.

*примерно так*